### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-13"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
    included_vehicle_type=[
        "van",
        "bus",
        "truck",
        "NaN",
        "pickup",
        "other",
        "sedan",
        "taxi",
        "motorbike",
        "suv",
        "hatchback",
    ],
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_SNAPSHOT]

# drop the one that lifetime is less than 4 seconds
# df_vehicle = df_vehicle.loc[(df_vehicle["lifetime"] > 4),]

# create a timestamp_unix at the end of lifespan
# df_vehicle["timestamp_unix_end"] = df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"].astype(np.int64))
# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

## Cross vs Vehicle

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

## Reverse vs Vehicle

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [4]:
df_vehicle["lifetime"]

2342    1.000
2335    1.240
2329    1.639
2330    1.399
2322    3.717
        ...  
41      1.360
32      2.239
33      0.599
34      0.480
25      1.120
Name: lifetime, Length: 223, dtype: float64

In [5]:
df_vehicle = match_reverse_to_vehicle(df_reverse, df_vehicle)
df_vehicle = match_cross_to_vehicle(
    df_cross,
    df_vehicle,
)


df_vehicle = df_vehicle[
    (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
]
df_vehicle = df_vehicle[
    [
        *conf.VEHICLE_INTEREST_SNAPSHOT,
        "cross_count",
        "reverse_count",
        "original_video_name",
        # "frame_no",
    ]
]

df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)
df_vehicle

,camera,timestamp_precise,xmin,xmax,ymax,lifetime,plate_number,vehicle_type,cross_count,reverse_count,original_video_name
1841,mbk-14-13-vehicle,2024-04-28 06:11:14.504,1061,1735,454,211.501,NaN,NaN,5,2,mbk-14-13-vehicle_1714284601.mp4
1825,mbk-14-13-vehicle,2024-04-28 06:18:02.074,1262,2077,473,84.296,ฮค9016,pickup,3,1,mbk-14-13-vehicle_1714284901.mp4
1826,mbk-14-13-vehicle,2024-04-28 06:18:34.186,1644,2110,463,5.279,สถ23,tuktuk,1,1,mbk-14-13-vehicle_1714284901.mp4
1807,mbk-14-13-vehicle,2024-04-28 06:19:16.894,1121,2182,456,73.303,NaN,NaN,21,1,mbk-14-13-vehicle_1714284901.mp4
1808,mbk-14-13-vehicle,2024-04-28 06:26:42.737,1269,2096,454,45.227,NaN,NaN,1,0,mbk-14-13-vehicle_1714285501.mp4
1784,mbk-14-13-vehicle,2024-04-28 06:29:51.245,1030,2079,477,115.251,NaN,NaN,5,2,mbk-14-13-vehicle_1714285501.mp4
1785,mbk-14-13-vehicle,2024-04-28 06:31:46.496,915,1847,449,111.370,NaN,NaN,2,0,mbk-14-13-vehicle_1714285801.mp4
1786,mbk-14-13-vehicle,2024-04-28 06:32:35.962,1173,2121,454,18.475,NaN,NaN,1,0,mbk-14-13-vehicle_1714285801.mp4
1765,mbk-14-13-vehicle,2024-04-28 06:35:01.365,1028,1840,510,6.162,NaN,NaN,0,0,mbk-14-13-vehicle_1714286101.mp4
1766,mbk-14-13-vehicle,2024-04-28 06:35:13.121,1335,1990,466,49.186,NaN,NaN,4,0,mbk-14-13-vehicle_1714286101.mp4


## TODO check cross walk angle?
